## **サンプル: ホテル予約のためのマルチAIエージェント**

今日の急速に変化する世界では、出張の計画は単に飛行機とホテルの予約をするだけではありません。調整と効率性のレベルが求められ、それを達成するのは難しいこともあります。ここでマルチAIエージェントが登場し、私たちの旅行ニーズの管理方法を革新します。

賢いエージェントのチームが手元にいて、あなたの旅行のあらゆる側面を正確かつ簡単に処理してくれると想像してみてください。高度なAI技術により、予約サービス用エージェントと旅程調整用エージェントを作成し、シームレスでストレスのない旅行体験を実現しています。

これは基本的なシナリオです。出張の計画では、航空券情報やホテル情報などを得るために出張手配担当者に相談する必要があります。AIエージェントを通じて、予約サービス用エージェントと旅程調整用エージェントを構築し、協力して知能レベルを向上させることができます。


# Azure AI エージェントサービスを初期化し、**.env** から構成情報を取得する

### **.env**

.env ファイルを作成します

**.env** には Azure AI エージェントサービスの接続文字列、AOAI で使用されるモデル、および対応する Google API サーチサービスの API、ENDPOINT などが含まれます。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "あなたの Azure AI エージェントサービスモデル展開名"

[**NOTE**] 100,000 トークン/分のレート制限および600リクエスト/分のレート制限があるモデルが必要です。

  Microsoft Foundry - モデルとエンドポイントでモデルを取得できます。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "あなたの Azure AI エージェントサービスプロジェクト接続文字列"

  AI Foundry ポータル画面のプロジェクト概要からプロジェクト接続文字列を取得できます。

- **SERPAPI_SEARCH_API_KEY** = "あなたの SERPAPI サーチ API キー"
- **SERPAPI_SEARCH_ENDPOINT** = "あなたの SERPAPI サーチ エンドポイント"

Azure AI エージェントサービスのモデル展開名とプロジェクト接続文字列を取得するには、Azure AI エージェントサービスを作成する必要があります。直接作成するには [このテンプレート](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) を使用することを推奨します（***注意:*** Azure AI エージェントサービスは現在、一部の地域でのみ設定されています。リージョンの設定については [こちらのリンク](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) を参照してください）

エージェントは SERPAPI にアクセスする必要があります。[このリンク](https://serpapi.com/searches) から登録することを推奨します。登録後、ユニークな API キーとエンドポイントを取得できます。


# Azure にログインする

次に Azure にログインする必要があります。VScode でターミナルを開き、`az login` コマンドを実行してください。


# Setup 

このノートブックを実行するには、以下のライブラリをインストールする必要があります。必要なライブラリとそれに対応するpipインストールコマンドのリストは次のとおりです。

azure-identity: Azure認証用。
requests: HTTPリクエストを行うため。
semantic-kernel: セマンティックカーネルフレームワーク用（これはカスタムまたは特定のライブラリであると想定されるため、特定のソースやリポジトリからインストールする必要があるかもしれません）。


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 説明: 
import asyncio: これはPythonで非同期プログラミングをサポートするasyncioモジュールをインポートします。asyncとawait構文を使って並行コードを書くことができます。 
from typing import Annotated: これはtypingモジュールからAnnotated型をインポートします。Annotatedは型ヒントにメタデータを追加するために使われ、検証、ドキュメント作成、ツール用などさまざまな目的に役立ちます。


In [ ]:
import asyncio,os
from typing import Annotated

# 説明：
from dotenv import load_dotenv と load_dotenv() を使用することで、設定情報や機密情報（APIキーやデータベースのURLなど）を .env ファイルで簡単に管理でき、ソースコードから分離してアプリケーションをより安全かつ設定しやすくすることができます。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 説明:

インポート文: from azure.identity.aio import DefaultAzureCredential: これはazure.identity.aioモジュールからDefaultAzureCredentialクラスをインポートします。モジュール名のaio部分は非同期操作向けであることを示しています。

DefaultAzureCredentialの目的: DefaultAzureCredentialクラスはAzure SDK for Pythonの一部です。Azureサービスに認証するためのデフォルトの方法を提供します。環境変数、マネージドID、Azure CLIの資格情報など、特定の順序で複数の方法を試みて認証を行います。

非同期操作: aioモジュールはDefaultAzureCredentialクラスが非同期操作をサポートしていることを示します。これはasyncioとともに使用して、ノンブロッキングの認証リクエストを実行できることを意味します。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 説明:
semantic_kernelパッケージからさまざまなモジュールとクラスをインポートしています。各インポートの内訳は次の通りです:

semantic_kernel.agentsからのAgentGroupChat: このクラスはAIエージェントのグループチャットに関する機能を処理します。  
semantic_kernel.agents.azure_aiからのAzureAIAgentとAzureAIAgentSettings

AzureAIAgent: このクラスはAzure AIサービスを利用するAIエージェントの作成と管理に使用されます。

AzureAIAgentSettings: このクラスはAzureAIAgentの設定を構成するために使用されます。  
semantic_kernel.agents.strategies.termination.termination_strategyからのTerminationStrategy:

このクラスは特定の条件下でAIエージェントの実行を終了させるための戦略を定義します。  
semantic_kernel.contents.chat_message_contentからのChatMessageContent:

このクラスはチャットメッセージの内容を扱うために使用されます。  
semantic_kernel.contents.utils.author_roleからのAuthorRole:

このクラスはチャットメッセージの文脈での著者の役割を定義します。  

semantic_kernel.functions.kernel_function_decoratorからのkernel_function: このデコレーターはセマンティックカーネルフレームワーク内で実行可能なカーネル関数を定義するために使用されます。  
これらのインポートは、ホテルの予約などのタスクのために、グループチャット環境で相互作用できるAIエージェントの作成と管理に必要なコンポーネントを設定します。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 説明:
次に、azure.ai.projects.models モジュールから CodeInterpreterTool クラスをインポートします。

CodeInterpreterTool: このクラスは Azure AI SDK の一部であり、AI プロジェクトのコンテキスト内でコードを解釈・実行するために使用されます。コードスニペットの実行、コードの分析、または AI ワークフロー内でのコード実行の統合に関する機能を提供します。  
このインポートは、動的にコードを解釈・実行するタスクに役立つ可能性がある CodeInterpreterTool をプロジェクトで利用するために必要なコンポーネントをセットアップします。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 説明: 
ApprovalTerminationStrategyクラスは、AIエージェントの動作を終了させる特定の戦略を提供します。エージェントは、その対話履歴の最後のメッセージに「saved」という単語が含まれている場合に終了します。これは、あるものが「保存された」という確認を受け取った時点でエージェントのタスクが完了と見なされるシナリオで役立つ可能性があります。対話方法を定義します。予約プランが保存された後、「saved」信号を受信した時点で停止することができます。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 説明：

このコード行は、create() メソッドを呼び出してデフォルトまたは事前定義された設定で AzureAIAgentSettings オブジェクトを初期化します。この設定オブジェクト（ai_agent_settings）は、AzureAIAgent インスタンスの構成や管理に使用できます。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 説明:
requestsライブラリをインポートすることで、簡単にHTTPリクエストを行い、Pythonコード内でウェブサービスとやり取りすることができます。


In [ ]:
import requests

# 説明:
これはSERP（検索エンジン結果ページ）APIサービスにアクセスするためのAPIキーを格納する変数です。APIキーは、あなたのアカウントに関連付けられたリクエストを認証するために使用される一意の識別子です。

'GOOGLE_SEARCH_API_KEY': これはプレースホルダーの文字列です。''GOOGLE_SEARCH_API_KEY'を実際のSERP APIキーに置き換える必要があります。

目的: この行の目的は、APIキーを変数に格納し、SERP APIサービスへのリクエストを認証するために使用できるようにすることです。APIキーはサービスにアクセスし、検索を実行するために必要です。

SERP APIキーの取得方法: SERP APIキーを取得するには、https://serpapi.com の一般的な手順に従ってください（具体的な手順は使用する特定のSERP APIサービスによって異なる場合があります）：

SERP APIサービスを選択する: SerpAPI、Google Custom Search JSON APIなど、いくつかのSERP APIサービスがあります。ニーズに最も合ったものを選択してください。

アカウント登録:

選択したSERP APIサービスのウェブサイト https://www.serpapi.com にアクセスし、アカウント登録を行います。基本情報の提供やメールアドレスの確認が必要な場合があります。

APIキーを作成する:

登録後、アカウントにログインし、APIセクションまたはダッシュボードに移動します。新しいAPIキーを作成または生成するオプションを探してください。
APIキーをコピーする:

APIキーが生成されたら、それをコピーします。このキーはSERP APIサービスへのリクエストを認証するために使用されます。
プレースホルダーを置き換える:

.envファイル内のプレースホルダーを置き換えてください。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 説明:
BookingPlugin クラスは、Serpapi.com の Google 検索 API を使用してホテルやフライトの予約を行うメソッドを提供します。必要なパラメータを構築し、API リクエストを送信し、レスポンスを処理して関連する予約情報を返します。API キー（SERPAPI_SEARCH_API_KEY）とエンドポイント（SERPAPI_SEARCH_ENDPOINT）は、Google 検索 API への認証およびリクエスト送信に使用されます。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 説明:
SavePlugin クラスは、Azure AI サービスを使用して旅行プランを保存する saving_plan メソッドを提供します。Azure の資格情報を設定し、AI エージェントを作成し、ユーザーの入力を処理して旅行プランの内容を生成および保存し、ファイルの保存やクリーンアップ操作を行います。このメソッドは正常に完了すると "Saved" を返します。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 説明：
このコードは、ユーザーの入力に基づいてフライトとホテルの予約、および旅行プランの保存を処理するAzure AIエージェントを設定します。Azureの資格情報を使用してエージェントを作成および構成し、グループチャットを通じてユーザー入力を処理し、タスク完了後に適切にクリーンアップを行います。エージェントはそれぞれのタスクを実行するために特定のプラグイン（BookingPluginおよびSavePlugin）を使用します。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されています。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる場合があります。原文の言語による文書が最も正確な情報源であることをご理解ください。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用によって生じた誤解や誤訳に関しましては、一切の責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
